In [1]:
import numpy as np
import matplotlib.pyplot as plt

Now we create different test data for computing the kernel sizes on.

In [2]:

def gaussian_disk():
    """
        Generate a random point in the unit disk using Gaussian sampling.
    """
    point = np.random.normal(loc=0.0, scale=1.0, size=2)
    norm_point = np.linalg.norm(point)
    scaled_point = point / norm_point * np.random.uniform(0, 1)**(1/2)
    return scaled_point


def gaussian_ball():
    """
        Generate a random point in the unit ball using Gaussian sampling.
    """
    point = np.random.normal(loc=0.0, scale=1.0, size=3)
    norm_point = np.linalg.norm(point)
    scaled_point = point / norm_point * np.random.uniform(0, 1)**(1/3)
    return scaled_point

def cutoff_and_rescale_noise(noise, cutoff_radius):
    """
    Cutoff noise vectors that exceed the given radius and rescale them.

    Arguments:
    - noise: (N, d) array of sampled noise vectors.
    - cutoff_radius: The threshold norm for noise.

    Returns:
    - Adjusted noise with norm constraint.
    """
    norms = np.linalg.norm(noise, axis=1)
    mask = norms > cutoff_radius  # Identify outliers

    # Rescale the noise vectors that exceed the cutoff
    noise[mask] = noise[mask] / norms[mask, np.newaxis] * cutoff_radius

    return noise

# Function to generate random points within a 3D ball of radius R
def random_points_in_ball(R, num_points, center = (0, 0, 0), dim=3):
    """
        Generate random points uniformly distributed in a ball of radius R in 'dim' dimensions.
    """
    points = []
    for _ in range(num_points):
        # generate a random point from a 3D Gaussian distribution
        point = gaussian_ball()
        # scale the point to be within the ball
        scaled_point = R*point
        translated_point = scaled_point + np.array(center)
        points.append(scaled_point)
    return np.array(points)

def construct_symmetric_ball(R, num_points, center=(0, 0, 0)):
    """
    Create a ball where every point explicitly includes its negation.
    """
    num_points_half = num_points // 2
    points = random_points_in_ball(R, num_points_half, center=center)
    return np.vstack((points, -points))

def symmetric_ball_shifted(R, num_points, new_center=(3, 3, 3)):
    """
    Generate a symmetric ball centered at the origin and then shift it to a new center.
    
    Parameters:
        R (float): Radius of the ball.
        num_points (int): Total number of points in the ball (including negated points).
        new_center (tuple): The new center to which the ball should be shifted.
    
    Returns:
        ndarray: An array of points representing the shifted symmetric ball.
    """
    # Generate half the points for the symmetric ball
    num_points_half = num_points // 2
    points = []
    
    for _ in range(num_points_half):
        # Generate a random point in the unit ball using Gaussian sampling
        point = np.random.normal(loc=0.0, scale=1.0, size=3)
        norm_point = np.linalg.norm(point)
        scaled_point = point / norm_point * np.random.uniform(0, 1)**(1/3)
        points.append(R * scaled_point)
    
    # Create the symmetric ball by adding negated points
    points = np.array(points)
    symmetric_points = np.vstack((points, -points))
    
    # Shift the ball to the new center
    shifted_points = symmetric_points + np.array(new_center)
    
    return shifted_points

def random_points_in_ball_with_noise(R, num_points, noise_std=0.1, dim=3):
    """
        Generate random points uniformly distributed in a ball of radius R in 'dim' dimensions
        and add Gaussian noise.
    """
    points = random_points_in_ball(R, num_points, dim)
    noise = np.random.normal(0, noise_std, size=points.shape)
    noisy_points = points + noise
    return noisy_points

def random_points_in_ball_with_cutoff_noise(R, num_points, noise_std=0.1, cutoff=0.2, dim=3):
    """
    Generate random points uniformly distributed in a ball of radius R in 'dim' dimensions
    and add Gaussian noise with a cutoff and rescaling.

    Arguments:
    - R: Radius of the ball.
    - num_points: Number of points to generate.
    - noise_std: Standard deviation of the noise.
    - cutoff: Maximum allowable norm for noise.
    - dim: Dimensionality of the space (default 3).

    Returns:
    - Noisy points inside the ball with constrained noise.
    """
    points = random_points_in_ball(R, num_points, dim=dim)
    noise = np.random.normal(0, noise_std, size=points.shape)
    
    # Apply cutoff and rescaling
    noise = cutoff_and_rescale_noise(noise, cutoff)

    noisy_points = points + noise
    return noisy_points


# Function to generate random points in the disk of radius R - 2d or 3d
def random_points_in_disk(R, num_points):
    """
        Generate random points within a 2D disk using Gaussian sampling.

        Arguments:
        - radius: The radius of the disk.
        - num_points: Number of random points to sample.

        Returns:
        - points: Randomly sampled points within the disk.
    """
    points = []

    for _ in range(num_points):
        # generate a random point from a 2D Gaussian distribution
        point = gaussian_disk()
        # scale the point to be within the disk
        scaled_point = R*point
        points.append(scaled_point)
    # Return points as an array of shape (num_points, 2)
    return np.array(points)

def random_points_in_disk_with_noise(R, num_points, noise_std=0.1):
    """
        Generate random points within a 2D disk and add Gaussian noise.
    """
    points = random_points_in_disk(R, num_points)
    noise = np.random.normal(0, noise_std, size=points.shape)
    noisy_points = points + noise
    return noisy_points

def random_points_in_disk_with_cutoff_noise(R, num_points, noise_std=0.1, cutoff=0.2):
    """
    Generate random points uniformly distributed in a disk of radius R in '2D' dimensions
    and add Gaussian noise with a cutoff and rescaling.

    Arguments:
    - R: Radius of the disk.
    - num_points: Number of points to generate.
    - noise_std: Standard deviation of the noise.
    - cutoff: Maximum allowable norm for noise.

    Returns:
    - Noisy points inside the disk with constrained noise.
    """
    points = random_points_in_disk(R, num_points)
    noise = np.random.normal(0, noise_std, size=points.shape)
    
    # Apply cutoff and rescaling
    noise = cutoff_and_rescale_noise(noise, cutoff)

    noisy_points = points + noise
    return noisy_points

def random_points_in_ellipse(a, b, num_points):
    """
        Generate random points within a 2D ellipse using Gaussian sampling.

        Arguments:
        - a: Semi-major axis length.
        - b: Semi-minor axis length.
        - num_points: Number of random points to sample.

        Returns:
        - points: Randomly sampled points within the ellipse.
    """
    points = []
    for _ in range(num_points):
        gaussian_point = gaussian_disk()
        ellipse_point = np.array([a*gaussian_point[0], b*gaussian_point[1]])
        points.append(ellipse_point)
    return np.array(points)

def random_points_in_ellipse_with_noise(a, b, num_points, noise_std=0.1):
    """
        Generate random points within a 2D ellipse and add Gaussian noise.
    """
    points = random_points_in_ellipse(a, b, num_points)
    noise = np.random.normal(0, noise_std, size=points.shape)
    noisy_points = points + noise
    return noisy_points

def random_points_in_ellipsoid(a, b, c, num_points):
    """
        Generate random points within a 3D ellipsoid using Gaussian sampling.

        Arguments:
        - a: Semi-principal axis along the x-axis.
        - b: Semi-principal axis along the y-axis.
        - c: Semi-principal axis along the z-axis.
        - num_points: Number of random points to sample.

        Returns:
        - points: Randomly sampled points within the ellipsoid.
    """
    points = []
    for _ in range(num_points):
        gaussian_point = gaussian_ball()
        ellipsoid_point = np.array([a*gaussian_point[0], b*gaussian_point[1], c*gaussian_point[2]])
        points.append(ellipsoid_point)
    return np.array(points)

def random_points_in_ellipsoid_with_noise(a, b, c, num_points, noise_std=0.1):
    """
        Generate random points within a 3D ellipsoid and add Gaussian noise.
    """
    points = random_points_in_ellipsoid(a, b, c, num_points)
    noise = np.random.normal(0, noise_std, size=points.shape)
    noisy_points = points + noise
    return noisy_points

def random_points_in_ellipsoid_with_cutoff_noise(a, b, c, num_points, noise_std=0.1, cutoff=0.2):
    """
    Generate random points in an ellipsoid and add Gaussian noise with cutoff and rescaling.

    Arguments:
    - a, b, c: Semi-axes of the ellipsoid.
    - num_points: Number of points to generate.
    - noise_std: Standard deviation of the noise.
    - cutoff: Maximum allowable norm for noise.

    Returns:
    - Noisy points inside the ellipsoid with constrained noise.
    """
    points = random_points_in_ellipsoid(a, b, c, num_points)
    noise = np.random.normal(0, noise_std, size=points.shape)
    
    # Apply cutoff and rescaling
    noise = cutoff_and_rescale_noise(noise, cutoff)

    noisy_points = points + noise
    return noisy_points

In [3]:
#subsample k points from array
def subsample_points(input_data, number_samples):
    """ Subsample k points from input data"""

In [4]:
# Function to apply matrix transformation A to points
def apply_forwardmodel(A, points):
    return np.dot(points, A.T)

def projection_nullspace(A, x):
    """
        Compute the projection of a point x onto the null space of A, i.e., P_{\mathcal{N}(A)}(x).
        This is equivalent to (I - A^dagger A) x.
    """
    A_dagger = np.linalg.pinv(A)
    return np.dot(np.eye(A.shape[1]) - np.dot(A_dagger, A), x)

<>:6: SyntaxWarning: invalid escape sequence '\m'
<>:6: SyntaxWarning: invalid escape sequence '\m'
/var/folders/93/0d4f5dmx4x580p8zm8nf2znr0000gp/T/ipykernel_36680/3471909342.py:6: SyntaxWarning: invalid escape sequence '\m'
  """


In [5]:
# Algorithm 1: Iterative Diameter Estimation
def diameters_feasiblesets_noiseless(A, input_data, target_data, p, epsilon=1e-2):
    """
        Implements the iterative algorithm for diameter estimation of the feasible set, consisting of all possible target data points, for each target point.
        Arguments:
        - A: The matrix (for which we are computing the Moore-Penrose inverse) of the inverse problem input_data = A(target_data)+noise.
        - input_data: Input data for an approximate inverse method.
        - target_data: Target or ground truth data for an approximate inverse method.
        - p: order of the norm, default p=2 for MSE computation.
        - epsilon: Noise level in the inverse problem input_data = A(target_data)+noise.

        Returns:
        - diameters: The estimated diameters of the feasible set, consisting of all possible target data points, for each target point.
    """

    # Compute Moore-Penrose-Inverse of A
    A_dagger = np.linalg.pinv(A)
    diameters = []
    max_diameters = 0

    for y in input_data:
        # prepare input data
        y = np.hstack((y, 0))  # extend to input dimension for compatibility with A
        x_perp_y = np.dot(A_dagger, y) # projection onto range of A
        diam_F_y = 0

        for point in target_data:
            # check if point satisfies the condition
            condition = np.linalg.norm(np.dot(np.dot(A_dagger, A), point - x_perp_y))
            if condition <= 2*epsilon:
                # print(f"Point satisfying condition found (ε = {epsilon})")
                # calculate projection of target point onto nullspace of A
                proj_nullspace = projection_nullspace(A, point)
                diam_x_n = 2*np.linalg.norm(proj_nullspace, ord = p)

                # update diameter if necessary
                if diam_x_n > diam_F_y:
                    diam_F_y = diam_x_n

        # Store final diameter for this input point
        diameters.append(diam_F_y)

    return diameters, max_diameters

In [ ]:
def diameters_feasiblesets(A, input_data, target_data, p=2, epsilon=1e-1):
    """
    Implements the iterative algorithm for diameter estimation of the feasible set, consisting of all possible target data points, for each target point.
        Arguments:
        - A: The matrix (for which we are computing the Moore-Penrose inverse) of the inverse problem input_data = A(target_data)+noise.
        - input_data: Input data for an approximate inverse method.
        - target_data: Target or ground truth data for an approximate inverse method.
        - p: order of the norm, default p=2 for MSE computation.
        - epsilon: Noise level in the inverse problem input_data = A(target_data)+noise.

        Returns:
        - diameters: dim(0)= shape(input_data), the estimated diameters of the feasible set, consisting of all possible target data points, for each target point.
    """
    # Compute Moore-Penrose-Inverse of F
    F = np.hstack((A, np.eye(A.shape[0])))  # Construct F: (A | I) : (3x6)

    # Step 2: Compute diameters
    diameter_means = []
    max_diameters = []

    for y in input_data:
        max_diam_F_y = 0

        for x_n in target_data:
            xcomp = len(x_n)
            #y_extended = np.hstack((y, 0))  # Extend y_i to 3D
            e_n = y - np.dot(A,x_n) # Compute noise vector

            if np.linalg.norm(e_n) <= epsilon:  # Check if noise is in E
                # Project onto the null space of F
                proj_nullspace = projection_nullspace(F, np.hstack((x_n, e_n)))[0:xcomp] # Project (x_n, e_n) onto nullspace of F, only take dim of x_n
                #print(np.shape(proj_nullspace))

                # Compute diameter based on projection
                diameter = 2 * np.linalg.norm(proj_nullspace, ord = p) 
                diameter_means.append(diameter)

                if diameter > max_diam_F_y:
                    max_diam_F_y = diameter

        max_diameters.append(max_diam_F_y)
        max_diameter = np.max(max_diameters)

    return diameters_means, max_diameter

In [7]:
def wc_kernelsize_oversamples(A, input_data, target_data, p, max_k, epsilon=1e-1):
    """
    Computes the worst-case kernel size under noise using Algorithm 2.

    Args:
        - A: The matrix (for which we are computing the Moore-Penrose inverse) of the inverse problem input_data = A(target_data)+noise.
        - input_data: Input data for an approximate inverse method.
        - target_data: Target or ground truth data for an approximate inverse method.
        - p: order of the norm, default p=2 for MSE computation.
        - max_k (int): Maximum number of samples from traget data.
        - epsilon: Noise level in the inverse problem input_data = A(target_data)+noise.

    Returns:
        Approximate worst-case kernel size for k_max and array of approximate worst-case kernel sizes for k in range(max_k)
    """

    #max_diameters = []
    wc_kersize = []

    for k in range(1,max_k,1):
        input_data_k = input_data[0:k,:]
        diameters, max_diameter = diameters_feasiblesets(A, input_data_k, target_data, p, epsilon)
        #max_diameters.append(max_diameter)
        print(f"For k={k} we get wc_kersize={max_diameter}")
        wc_kersize.append(max_diameter)
        #k = k+1
    wc_kersizef=np.max(wc_kersize)  
    
    return wc_kersizef, wc_kersize


In [8]:
def av_kernelsize_oversamples(A, input_data, target_data, p, max_k, epsilon=1e-1):
    """
    Computes the average kernel size under noise using Algorithm 2.

    Args:
        - A: The matrix (for which we are computing the Moore-Penrose inverse) of the inverse problem input_data = A(target_data)+noise.
        - input_data: Input data for an approximate inverse method.
        - target_data: Target or ground truth data for an approximate inverse method.
        - p: order of the norm, default p=2 for MSE computation.
        - max_k (int): Maximum number of samples from traget data.
        - epsilon: Noise level in the inverse problem input_data = A(target_data)+noise.

    Returns:
        Approximate average kernel size for k_max and array of approximate average kernel sizes for k in range(max_k)
    """

    av_kersizes = []

    for k in range(1,max_k,1):
        input_data_k = input_data[0:k,:]
        diameters, max_diameter = diameters_feasiblesets(A, input_data_k, target_data, p, epsilon)
        #max_diameters.append(max_diameter)
        av_kersize =  np.power(np.mean(np.power(diameters,p)), 1/p)
        av_kersizes.append(av_kersize)
        print(f"For k={k} we get av_kersize={av_kersize}")
        #k = k+1 
    av_kersize = av_kersizes[-1]
    
    return av_kersize, av_kersizes


In [9]:
def plot_wckersize_conv(wc_kersizef, kersize_approxis, ker_size, max_k):

    # Plot results
    x_axis = np.arange(2, max_k+1)
    y_axis = np.array(kersize_approxis)
    plt.plot(x_axis, y_axis)
    plt.axhline(ker_size, color='r')
    plt.ylim(0, ker_size + 0.1 * ker_size)
    plt.grid()
    plt.xlabel("Number of samples")
    plt.ylabel("Approximate  wc kernel size")
    plt.title("Number of Samples vs wc Kernel Size")
    plt.show()

    #max_diameter_total = max(max_diameters)
    print(f"Total Max Kernel Size: { wc_kersizef}")
    print(f"Analytical Kernel Size: {ker_size}")
    rel_error = (wc_kersizef - ker_size) / ker_size
    print(f"Relative Error: {rel_error}")

In [10]:
def plot_avkersize_conv(av_kersize, av_kersizes, ker_size, max_k):

    # Plot results
    x_axis = np.arange(2, max_k+1)
    y_axis = np.array(av_kersizes)
    plt.plot(x_axis, y_axis)
    plt.axhline(ker_size, color='r')
    plt.ylim(0, ker_size + 0.1 * ker_size)
    plt.grid()
    plt.xlabel("Number of samples")
    plt.ylabel("Approximate  wc kernel size")
    plt.title("Number of Samples vs wc Kernel Size")
    plt.show()

    #max_diameter_total = max(max_diameters)
    print(f"Total Max Kernel Size: {av_kersize}")
    print(f"Analytical Kernel Size: {ker_size}")
    rel_error = (av_kersize - ker_size) / ker_size
    print(f"Relative Error: {rel_error}")

In [ ]:
# check if P_{N(A)}(M_1) of first toy example set is symmetric

# Toy example 1

R = 2  # Radius of the ball
num_points = 1000  # Number of points in the set M_1
A = np.diag([1, 1, 0])  # Transformation matrix
#analytical kersize
ker_size = 2*R
av_ker_size = 2.8944 # icky integrals
p=2

# Generate points in B_R(0)
target_data = random_points_in_ball(R, num_points)
input_data =  apply_forwardmodel(A, target_data)

# Set the range of k values
max_k = 100

#wc_kersizef, kersize_approxis = wc_kernelsize_oversamples(A, input_data, target_data, p, max_k, epsilon=1e-1)
#plot_wckersize_conv(wc_kersizef, kersize_approxis, ker_size, max_k)

av_kersize, av_kersizes = av_kernelsize_oversamples(A, input_data, target_data, p, max_k, epsilon=1e-1)
plot_avkersize_conv(av_kersize, av_kersizes, av_ker_size, max_k)

For k=1 we get av_kersize=3.070682907442591
For k=2 we get av_kersize=2.5861683901822237
For k=3 we get av_kersize=2.4424010920796686
For k=4 we get av_kersize=2.4324791420348157
For k=5 we get av_kersize=2.462080531736495
For k=6 we get av_kersize=2.368618482620104
For k=7 we get av_kersize=2.36927165944643
For k=8 we get av_kersize=2.3791720853304654
For k=9 we get av_kersize=2.350756896882608
For k=10 we get av_kersize=2.340394520205966
For k=11 we get av_kersize=2.302047145532494
For k=12 we get av_kersize=2.31035378474989
For k=13 we get av_kersize=2.2869082125879765
For k=14 we get av_kersize=2.2508856133142476
For k=15 we get av_kersize=2.2204388143687575
For k=16 we get av_kersize=2.2249096123988883
For k=17 we get av_kersize=2.22293566140291
For k=18 we get av_kersize=2.2188550702464007
For k=19 we get av_kersize=2.215248991606639
For k=20 we get av_kersize=2.2066210473314736
For k=21 we get av_kersize=2.1916640410832238
For k=22 we get av_kersize=2.20039449844739
For k=23 we 

In [12]:
np.linalg.norm([1,2,3], ord = 2)

3.7416573867739413